In [4]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
from google import genai
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
import platform

In [5]:
load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')
google_key = os.getenv('GOOGLE_API_KEY')

In [6]:
openai = OpenAI()

openai_model = 'o3-mini'
google_model = 'gemini-2.5-flash'

In [7]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [8]:
def user_prompt(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [9]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt(python)}
    ]

In [10]:
def write_output_gpt(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized_gpt.cpp", "w") as f:
        f.write(code)

In [11]:
def write_output_google(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized_google.cpp", "w") as f:
        f.write(code)


In [12]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=openai_model, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output_gpt(reply)

In [13]:
def optimize_google(python):

    client = genai.Client(api_key=google_key)
    response = ""
    for chunk in client.models.generate_content_stream(
        model=google_model,contents=user_prompt(python)):
        print(chunk.text,end='')
        response+= chunk.text
    write_output_google(response)

In [14]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [15]:
exec(pi)

Result: 3.141592658589
Execution Time: 18.210901 seconds


In [ ]:
optimize_gpt(pi)

In [ ]:
optimize_google(pi)

In [18]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [19]:
exec(python_hard)


Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 75.602117 seconds


In [ ]:
optimize_gpt(python_hard)

In [ ]:
optimize_google(python_hard)

In [22]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=openai_model, messages=messages_for(python), stream=True)
    reply = ''
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')


In [23]:
def stream_google(python):
    client = genai.Client(api_key=google_key)
    stream = client.models.generate_content_stream(
        model=google_model,
        contents=user_prompt(python)
    )

    reply = ''
    for chunk in stream:
        chunk = chunk.text
        reply += chunk
        yield reply.replace('```cpp\n', '').replace('```', '')


In [24]:
def optimize(python, model):
    if model == 'GPT':
        result = stream_gpt(python)
    elif model == 'GOOGLE':
        result = stream_google(python)
    else:
        raise ValueError('Unknown Model')

    for item in result:
        yield item


In [ ]:
with gr.Blocks() as ui :
    with gr.Row():
        python = gr.Textbox(label="Python Code; ",lines=10,value=python_hard)
        cpp = gr.Textbox(label='C++ Code:',lines=10)
    with gr.Row():
        model = gr.Dropdown(['GPT','GOOGLE'],label='Select Model',value='GPT')
        convert = gr.Button('Convert Code')

    convert.click(optimize,inputs=[python,model],outputs=[cpp])

ui.launch(inbrowser=True)